In [28]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        pass
print('hello')

hello


In [29]:
!pip install torch torchvision albumentations

In [30]:
import torch
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

# Load pre-trained Faster R-CNN model
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)

# Modify the classifier for custom dataset
num_classes = 4  # Example: 3 classes + 1 background
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [31]:
import os
import numpy as np
import torch
from PIL import Image
import torchvision.transforms as T
from torch.utils.data import Dataset
import xml.etree.ElementTree as ET
from torch.utils.data import DataLoader



In [32]:
import torchvision.transforms as T

class CustomDataset(Dataset):
    def __init__(self, root, transforms=None):
        self.root = root
        self.transforms = transforms if transforms else T.ToTensor()  # Convert to tensor by default
        self.imgs = sorted(os.listdir(os.path.join(root, "images")))
        self.annotations = sorted(os.listdir(os.path.join(root, "annotations")))

    def load_annotations(self, ann_path):
        tree = ET.parse(ann_path)
        root = tree.getroot()

        boxes = []
        labels = []
        label_map = {"with_mask": 1, "without_mask": 2, "mask_weared_incorrect": 3}  # Modify as needed

        for obj in root.findall("object"):
            name = obj.find("name").text
            labels.append(label_map[name])  

            bndbox = obj.find("bndbox")
            xmin = int(bndbox.find("xmin").text)
            ymin = int(bndbox.find("ymin").text)
            xmax = int(bndbox.find("xmax").text)
            ymax = int(bndbox.find("ymax").text)

            boxes.append([xmin, ymin, xmax, ymax])

        return torch.tensor(boxes, dtype=torch.float32), torch.tensor(labels, dtype=torch.int64)

    def __getitem__(self, idx):
        img_path = os.path.join(self.root, "images", self.imgs[idx])
        annotation_path = os.path.join(self.root, "annotations", self.annotations[idx])

        image = Image.open(img_path).convert("RGB")
        boxes, labels = self.load_annotations(annotation_path)

        target = {"boxes": boxes, "labels": labels}

        # Convert image to tensor
        image = self.transforms(image)

        return image, target

    def __len__(self):
        return len(self.imgs)

In [33]:
import torchvision.transforms as transforms

transform = transforms.Compose([
    transforms.Resize((300, 400)),  # Resize all images to 300x400
    transforms.ToTensor(),  # Convert to tensor
])

In [34]:
# Apply the transformation when loading the dataset
dataset = CustomDataset("/kaggle/input/face-mask-detection", transforms=transform)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=2, shuffle=True)

In [35]:
dataset = CustomDataset("/kaggle/input/face-mask-detection")
image, target = dataset[0]

print(f"Image Size: {image.size}")
print(f"Target: {target}") 

Image Size: <built-in method size of Tensor object at 0x7ca8a5571210>
Target: {'boxes': tensor([[ 79., 105., 109., 142.],
        [185., 100., 226., 144.],
        [325.,  90., 360., 141.]]), 'labels': tensor([2, 1, 2])}


In [36]:
from sklearn.model_selection import train_test_split

# Load full dataset
dataset = CustomDataset("/kaggle/input/face-mask-detection")

# Generate train-test split indices
indices = list(range(len(dataset)))
train_indices, test_indices = train_test_split(indices, test_size=0.2, random_state=42)

# Create subsets
train_dataset = torch.utils.data.Subset(dataset, train_indices)
test_dataset = torch.utils.data.Subset(dataset, test_indices)

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True, collate_fn=lambda x: tuple(zip(*x)))
test_loader = DataLoader(test_dataset, batch_size=4, shuffle=False, collate_fn=lambda x: tuple(zip(*x)))

In [37]:
import torch.optim as optim

In [38]:
# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=0.0)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=0.0)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=0.0)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=0.0)
          (relu): ReLU(

In [39]:
# Define optimizer
learning_rate = 0.005  # Try different values (e.g., 0.001, 0.01)
optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9, weight_decay=0.0005)

In [40]:
from torch.utils.data import DataLoader

In [41]:
# Training loop
num_epochs = 5  # Try different values
for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0

    for images, targets in train_loader:
        images = [img.to(device) for img in images]  # Now img is a tensor
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

        optimizer.zero_grad()
        loss_dict = model(images, targets)
        loss = sum(loss for loss in loss_dict.values())

        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}")

if we try for 0.001 learning rate and for 7 epochs than result is look like

In [42]:
# Define optimizer
learning_rate = 0.001  # Try different values (e.g., 0.001, 0.01)
optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9, weight_decay=0.0005)

# Training loop
num_epochs = 7  # Try different values
for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0

    for images, targets in train_loader:
        images = [img.to(device) for img in images]  # Now img is a tensor
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

        optimizer.zero_grad()
        loss_dict = model(images, targets)
        loss = sum(loss for loss in loss_dict.values())

        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}")

Epoch [1/7], Loss: 90.2359
Epoch [2/7], Loss: 56.8272
Epoch [3/7], Loss: 48.6230
Epoch [4/7], Loss: 44.7022
Epoch [5/7], Loss: 40.4689
Epoch [6/7], Loss: 37.1928
Epoch [7/7], Loss: 35.0686
